In [ ]:
import pandas as pd
import numpy as np

from utils import column_name_mapping

In [2]:
df = pd.read_csv('data/Case Rigshospitalet.csv')

C:\Users\balda\AppData\Local\Temp\ipykernel_6556\4698354.py:1: DtypeWarning: Columns (9,13,22) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/Case Rigshospitalet.csv')


In [3]:
df.columns

Index(['Patient ID', 'Patientkontakt ID',
       'Patient alder på kontaktstart tidspunkt', 'Kontakt startdato',
       'Kontakt slutdato', 'Kontakttype', 'Indlæggelsesmåde',
       'Patientkontakttype', 'Aktionsdiagnosekode', 'Bidiagnosekode',
       'Behandlingsansvarlig Afdeling', 'Kontakt varighed (timer)',
       'Procedure-kode', 'Procedure-tillægskoder', 'Procedure udført',
       'Behandlingskontakt ID', 'Besøgstype', 'Patient køn',
       'Patient civilstand', 'Patient oprettet på Min SP (J/N)',
       'Patient land', 'Patient region', 'Patient postnummer',
       'Patient kommune'],
      dtype='object')

In [4]:
# renaming columns to english
df = df.rename(columns=column_name_mapping)

In [5]:
df.columns

Index(['patientID', 'patientContactID', 'patientAgeAtContactStart',
       'contactStartDate', 'contactEndDate', 'contactType',
       'appointmentUrgency', 'patientContactType', 'primaryDiagnosisCode',
       'secondaryDiagnosisCode', 'responsibleDepartment',
       'contactDurationHours', 'procedureCode', 'procedureSupplementaryCodes',
       'procedurePerformed', 'treatmentContactID', 'visitType',
       'patientGender', 'patientMaritalStatus', 'patientRegisteredInMySP',
       'patientCountry', 'patientRegion', 'patientPostalCode',
       'patientMunicipality'],
      dtype='object')

In [6]:
# procedureSupplementaryCodes and visitType have to many missing values
# rest does not seem useful (at least for now)
drop_columns = ['responsibleDepartment', 'secondaryDiagnosisCode', 'procedureSupplementaryCodes', 
                'visitType', 'patientRegisteredInMySP', 'patientCountry', 'patientRegion',
                'patientPostalCode', 'patientMunicipality','patientMaritalStatus','procedurePerformed',
                'contactDurationHours']

In [7]:
# filtering out missing patientID (46 cases) and droping redundant columns
df_cleaned = df.copy()
df_cleaned = df_cleaned[~df_cleaned['patientID'].isnull()].drop(columns=drop_columns)

In [8]:
df_cleaned.columns

Index(['patientID', 'patientContactID', 'patientAgeAtContactStart',
       'contactStartDate', 'contactEndDate', 'contactType',
       'appointmentUrgency', 'patientContactType', 'primaryDiagnosisCode',
       'procedureCode', 'treatmentContactID', 'patientGender'],
      dtype='object')

In [9]:
# converting to datetime
df_cleaned['contactStartDate'] = df_cleaned['contactStartDate'].str.replace(',', '.')
df_cleaned['contactEndDate'] = df_cleaned['contactEndDate'].str.replace(',', '.')

df_cleaned['contactStartDate'] = pd.to_datetime(df_cleaned['contactStartDate'])
df_cleaned['contactEndDate'] = pd.to_datetime(df_cleaned['contactEndDate'])

df_cleaned['contactDurationMin'] = (df_cleaned['contactEndDate'] - df_cleaned['contactStartDate']).dt.total_seconds() / 60

In [10]:
# few patient level columns at this stage (passed later to df_patient)
df_cleaned['numVisits'] = df_cleaned.groupby('patientID')['patientContactID'].transform('nunique')
df_cleaned['numProcedures'] = df_cleaned.groupby('patientID')['procedureCode'].transform('nunique')
df_cleaned['numContacts'] = df_cleaned.groupby('patientID')['treatmentContactID'].transform('nunique')

In [11]:
# deduplicate appointments at this point to ease further calculations
df_appointments = df_cleaned.drop_duplicates(subset='patientContactID', keep='first')
print('Num of rows/appointments after dedup: ', len(df_appointments))
df_appointments.head()

Num of rows/appointments after dedup:  1821183


,patientID,patientContactID,patientAgeAtContactStart,contactStartDate,contactEndDate,contactType,appointmentUrgency,patientContactType,primaryDiagnosisCode,procedureCode,treatmentContactID,patientGender,contactDurationMin,numVisits,numProcedures,numContacts
0,Z4115614,1301023075465,0,2023-01-13 12:32:00,2023-01-13 13:02:00,Fysisk fremmøde,Planlagt,2 Ambulant,DZ001,ZZ0151,176.794.528,Mand,30.0,6,6,6
1,Z4115614,1301023121831,0,2023-01-15 13:27:00,2023-01-15 13:57:00,Fysisk fremmøde,Planlagt,2 Ambulant,DZ001,ZZ0151,177.035.032,Mand,30.0,6,6,6
2,Z4119225,1301023357249,0,2023-01-30 10:20:00,2023-01-30 10:50:00,Fysisk fremmøde,Planlagt,2 Ambulant,DZ001,ZZ0151,178.269.449,Mand,30.0,6,3,6
3,Z4127033,1301023443807,0,2023-02-04 12:46:00,2023-02-04 13:16:00,Fysisk fremmøde,Planlagt,2 Ambulant,DZ001,ZZ0151,178.703.583,Mand,30.0,17,11,17
4,Z4127033,1301023443875,0,2023-02-05 12:22:00,2023-02-05 12:52:00,Fysisk fremmøde,Planlagt,2 Ambulant,DZ001,ZZ0151,178.703.779,Mand,30.0,17,11,17


In [13]:
df_appointments['cumulativeVisitDuration'] = df_appointments.groupby(['patientID'])['contactDurationMin'].transform('sum')
df_appointments['Age'] = df_appointments.groupby(['patientID'])['patientAgeAtContactStart'].transform('mean')
df_appointments['Gender'] = df_appointments['patientGender'].map({'Kvinde': 0}).fillna(1)

# wrapping diagnosis into set for each patient
diagnosis_set = df_appointments.groupby('patientID')['primaryDiagnosisCode'].agg(set).reset_index()
df_appointments = df_appointments.merge(diagnosis_set, on='patientID', how='left', suffixes=('', '_unique'))
df_appointments.rename(columns={'primaryDiagnosisCode_unique': 'Diagnosis'}, inplace=True)

# calculate number of contact types per patient
df_contact_counts = pd.crosstab(df_appointments['patientID'], df_appointments['contactType'])
df_contact_counts.columns = [f'num_{contact_type}' for contact_type in df_contact_counts.columns]
df_appointments = df_appointments.merge(df_contact_counts, on='patientID', how='left')

# calculate number of contact types per patient
df_urgency_counts = pd.crosstab(df_appointments['patientID'], df_appointments['appointmentUrgency'])
df_urgency_counts.columns = [f'num_{urgency_type}' for urgency_type in df_urgency_counts.columns]
df_appointments = df_appointments.merge(df_urgency_counts, on='patientID', how='left')

# calculate number of patient contact types per patient
df_patient_contact_type_counts = pd.crosstab(df_appointments['patientID'], df_appointments['patientContactType'])
df_patient_contact_type_counts.columns = [f'num_{patient_contact_type}' for patient_contact_type in df_patient_contact_type_counts.columns]
df_appointments = df_appointments.merge(df_patient_contact_type_counts, on='patientID', how='left')

C:\Users\balda\AppData\Local\Temp\ipykernel_6556\197153596.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_appointments['cumulativeVisitDuration'] = df_appointments.groupby(['patientID'])['contactDurationMin'].transform('sum')
C:\Users\balda\AppData\Local\Temp\ipykernel_6556\197153596.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_appointments['Age'] = df_appointments.groupby(['patientID'])['patientAgeAtContactStart'].transform('mean')
C:\Users\balda\AppData\Local\Temp\ipykernel_6556\19715359

In [ ]:
diagnosis_set = df_appointments.groupby('patientID')['primaryDiagnosisCode'].agg(set).reset_index()
df_appointments = df_appointments.merge(diagnosis_set, on='patientID', how='left', suffixes=('', '_unique'))
df_appointments.rename(columns={'primaryDiagnosisCode_unique': 'Diagnosis'}, inplace=True)

In [14]:
rename_columns = {
'num_Fysisk fremmøde':'PhysicalAppointments',
'num_Udekontakt':'HomeAppointments',
'num_Virtuel pt.kt.':'VirtualAppointments',
'num_2 Ambulant':'numRegularOutpatient',
'num_3 Akut Ambulant':'numAcuteAmbulatory',
'num_4 Ambulant Us/Op':'numUnderObservation',
'num_Akut':'numAcute',
'num_Planlagt':'numPlanned'
}

df_appointments = df_appointments.rename(columns=rename_columns)

In [15]:
df_appointments.columns

Index(['patientID', 'patientContactID', 'patientAgeAtContactStart',
       'contactStartDate', 'contactEndDate', 'contactType',
       'appointmentUrgency', 'patientContactType', 'primaryDiagnosisCode',
       'procedureCode', 'treatmentContactID', 'patientGender',
       'contactDurationMin', 'numVisits', 'numProcedures', 'numContacts',
       'cumulativeVisitDuration', 'Age', 'Gender', 'Diagnosis',
       'PhysicalAppointments', 'HomeAppointments', 'VirtualAppointments',
       'numAcute', 'numPlanned', 'numRegularOutpatient', 'numAcuteAmbulatory',
       'numUnderObservation'],
      dtype='object')

In [16]:
patient_columns = [
    'patientID', 'numVisits', 'numProcedures', 'numContacts',
    'cumulativeVisitDuration', 'Age', 'Gender', 'Diagnosis',
    'PhysicalAppointments', 'HomeAppointments', 'VirtualAppointments',
    'numAcute', 'numPlanned', 'numRegularOutpatient', 'numAcuteAmbulatory',
    'numUnderObservation'
]

df_patient = df_appointments[patient_columns]

In [17]:
df_patient = df_patient.drop_duplicates(subset='patientID', keep='first')
print('Num of rows/appointments after dedup: ', len(df_patient))
df_patient.head()

Num of rows/appointments after dedup:  331762


,patientID,numVisits,numProcedures,numContacts,cumulativeVisitDuration,Age,Gender,Diagnosis,PhysicalAppointments,HomeAppointments,VirtualAppointments,numAcute,numPlanned,numRegularOutpatient,numAcuteAmbulatory,numUnderObservation
0,Z4115614,6,6,6,342.0,0.0,1.0,"{DZ001, DQ540, DQ549}",5,0,1,0,6,6,0,0
2,Z4119225,6,3,6,260.0,0.0,1.0,{DZ001},6,0,0,0,6,6,0,0
3,Z4127033,17,11,17,1547.0,0.0,1.0,"{DZ001, DZ033A, DP599, DQ753, DK628, DZ135C}",14,1,2,0,17,17,0,0
5,Z4139519,3,3,3,120.0,0.0,1.0,{DZ001},3,0,0,0,3,3,0,0
7,Z4146746,4,3,4,180.0,0.0,1.0,{DZ001},3,1,0,0,4,4,0,0


In [ ]:
# save to csv
df_patient.to_csv('data/patient_dataframe.csv', index=False)